In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Layer, Input, Conv2D, Conv2DTranspose, Flatten, Dense, LeakyReLU, BatchNormalization, ReLU, Activation, Reshape, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from google.colab import drive

drive.mount('/content/drive')

print("GPU Available: ", tf.test.is_gpu_available())
print("Devices:", tf.config.list_physical_devices())



Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
GPU Available:  True
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import os
import shutil
from tqdm import tqdm
import signal
from contextlib import contextmanager
import time

# Setting up a timeout handler using signals
class TimeoutException(Exception):
    pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

# Function to copy a file with a timeout
def copy_file_with_timeout(src, dst, timeout=10):
    try:
        with time_limit(timeout):
            shutil.copy(src, dst)
        return True
    except TimeoutException as e:
        print(f"Timeout occurred for {os.path.basename(src)}: {e}")
        return False

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the source and destination paths
source_path = '/content/drive/My Drive/GANProject/spec_512'
destination_path = '/content/GANProject/spec512'

# Ensure the destination directory exists
os.makedirs(destination_path, exist_ok=True)

# List all npy files and copy them with a timeout and retry if needed
files = [f for f in os.listdir(source_path) if f.endswith('.npy')]
for file in tqdm(files, desc="Copying files"):
    src_file = os.path.join(source_path, file)
    dest_file = os.path.join(destination_path, file)
    if not os.path.exists(dest_file):  # 이미 파일이 존재하지 않는 경우에만 복사
      success = copy_file_with_timeout(src_file, dest_file, timeout=30)  # Set timeout to 30 seconds
      if not success:
          drive.mount('/content/drive', force_remount= True)
          print(f"Retrying {file}...")
          time.sleep(5)  # Wait for 5 seconds before retrying
          copy_file_with_timeout(src_file, dest_file, timeout=30)  # Corrected function call for retry

print("Copy complete")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Copying files: 100%|██████████| 21172/21172 [18:03<00:00, 19.54it/s]

Copy complete


In [ ]:
def load_spectrogram(file_path):
    # EagerTensor에서 바로 numpy 배열로 변환 후 decode
    file_path = file_path.numpy().decode()
    spec = np.load(file_path)
    spec = np.expand_dims(spec, axis=-1)  # 채널 차원 추가
    factor = np.max(spec) - np.min(spec)
    if factor > 0:
        spec = 2 * (spec - np.min(spec)) / factor - 1  # -1과 1 사이로 정규화
    return spec

def preprocess(file_path):
    # TensorFlow ops에서 사용할 수 있도록 load_spectrogram을 wrapper 함수로 호출
    spectrogram = tf.py_function(load_spectrogram, [file_path], Tout=[tf.float32])
    spectrogram = spectrogram[0]
    return spectrogram

def create_dataset(data_dir, batch_size=32):
    # 파일 경로 로드
    file_paths = [os.path.join(data_dir, fname) for fname in os.listdir(data_dir) if fname.endswith('.npy')]
    print("file_paths")
    # TensorFlow Dataset 객체 생성
    dataset = tf.data.Dataset.from_tensor_slices(file_paths)
    print("tensor_slice")
    # 파일 경로를 실제 데이터로 변환
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    print("dataset.map")
    # 배치 처리 및 prefetching
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    print("dataset.batch")
    # 데이터셋의 배치 형태를 확인하기 위한 코드
    return dataset

In [ ]:
def build_generator(noise_shape):
    #input size: 1024
    inputs = Input(shape=noise_shape)

    x = Dense(32*32*32)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Reshape((32, 32, 32))(x)
    # Upsampling
    x = Conv2DTranspose(256, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #64*64*256
    x = Conv2DTranspose(128, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #128*128*128
    x = Conv2DTranspose(64, kernel_size=4, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #256*256*64
    x = Conv2DTranspose(1, kernel_size=4, strides=2, padding='same')(x)
    #512*512*1
    x = Activation('tanh')(x)

    return Model(inputs, x)

def build_discriminator(input_shape):
    #input size: 512*512
    inputs = Input(shape=input_shape)

    x = Conv2D(1, kernel_size=5, strides=2, padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    x = RandomShuffleLayer()(x)

    #256*256*64
    x = Conv2D(16, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = RandomShuffleLayer()(x)
    '''
    #128*128*128
    x = Conv2D(256, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = RandomShuffleLayer()(x)
    '''
    x= Flatten()(x)
    x= Dense(64)(x)
    x = LeakyReLU(alpha=0.2)(x)
    #64*64*256
    x = Dense(1)(x)
    x = Activation('sigmoid')(x)

    return Model(inputs, x)


class RandomShuffleLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(RandomShuffleLayer, self).__init__(**kwargs)

    def call(self, inputs):
        # Assuming inputs are of shape [batch_size, height, width, channels]
        def shuffle_along_axis(image):
            # Shuffle along the width (axis=1)
            shuffled = tf.random.shuffle(tf.transpose(image, [1, 0, 2]))
            return tf.transpose(shuffled, [1, 0, 2])

        # Apply to each image in the batch
        outputs = tf.map_fn(shuffle_along_axis, inputs, dtype=tf.float32)
        return outputs


In [ ]:
generator = build_generator((100,))

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

In [ ]:
import matplotlib.pyplot as plt

def save_spectrogram(spectrogram, epoch, output_dir, X_mean=-5, X_std=45):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Convert EagerTensor to numpy if not already
    if isinstance(spectrogram, tf.Tensor):
        spectrogram = spectrogram.numpy()

    # Squeeze to remove single-dimensional entries from the shape
    spectrogram = spectrogram.squeeze()

    # Reversing the normalization formula to get the original log magnitude
    X_mag = (spectrogram * X_std) + X_mean
    np.save(os.path.join(output_dir, f'spectrogram_epoch_{epoch}.npy'), X_mag)

    # Configure the plot
    plt.figure(figsize=(10, 4))
    plt.imshow(X_mag, aspect='auto', origin='lower', cmap='viridis')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f'Spectrogram at Epoch {epoch}')
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.tight_layout()

    # Save the figure
    plt.savefig(os.path.join(output_dir, f'spectrogram_epoch_{epoch}.png'))
    plt.close()


In [ ]:
np.set_printoptions(linewidth=200)

@tf.function

def train_step(images, valid, fake, generator, discriminator, combined, noise_shape):
    noise = tf.random.normal([tf.shape(images)[0]]+ noise_shape)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Calculate losses
        disc_loss_real = tf.keras.losses.binary_crossentropy(valid[:tf.shape(real_output)[0]], real_output, from_logits=False)
        disc_loss_fake = tf.keras.losses.binary_crossentropy(fake[:tf.shape(fake_output)[0]], fake_output, from_logits=False)
        disc_loss = (disc_loss_real + disc_loss_fake) / 2

        # Calculate probabilities for real and fake
        real_prob = tf.math.reduce_mean(real_output)
        fake_prob = tf.math.reduce_mean(fake_output)

        #tf.print("real_output: ", real_prob, summarize=-1)
        #tf.print("fake_output: ", fake_prob, summarize=-1)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator.optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator.optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))


    #tf.print("gradients_of_discriminator: ", gradients_of_discriminator, summarize=-1)
    #tf.print("Discriminator weights: ", [var.name for var in discriminator.trainable_variables])

    return disc_loss, gen_loss, real_prob, fake_prob

import traceback

def train_gan(generator, discriminator, combined, dataset, epochs, batch_size, noise_shape, save_interval, output_dir, start_epoch):
    noise_shape = list(noise_shape)
    print("train_gan started")
    loss_log_path = os.path.join(output_dir, "training_log.txt")
        # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    valid = np.ones((batch_size, 1)) * 0.9 # Label Smoothing 적용
    fake = np.zeros((batch_size, 1))

    with open(loss_log_path, "w") as file:
        file.write("Start of training\n")

    try:
        for epoch in range(start_epoch, epochs):
            print(f"Epoch {epoch+1} started")
            for batch_index, data in enumerate(dataset):
                imgs = data[0] if isinstance(data, tuple) else data
                if tf.shape(imgs)[0] != batch_size:
                    continue  # Skip incomplete batches
                d_loss, g_loss, real_prob, fake_prob = train_step(imgs, valid, fake, generator, discriminator, combined, noise_shape)
                print(".", end="")
                if (batch_index + 1) % 80 == 0:
                    print("")
                    log_message = f"Epoch {epoch + 1}, Batch {batch_index + 1}\nD loss: {d_loss.numpy()}, \nG loss: {g_loss.numpy()}\nReal prob: {real_prob.numpy()*100:.2f}%, Fake prob: {fake_prob.numpy()*100:.2f}%\n"
                    print(log_message)
                    with open(loss_log_path, "a") as file:
                        file.write(log_message)


            if (epoch + 1) % save_interval == 0:
                generator.save(f'{output_dir}/generator_epoch_{epoch + 1}')
                discriminator.save(f'{output_dir}/discriminator_epoch_{epoch + 1}')
                noise = tf.random.normal([batch_size]+noise_shape)
                generated_spec= generator(noise, training=False)[0]
                save_spectrogram(generated_spec, epoch+1, output_dir)

            with open(loss_log_path, "a") as file:
                file.write(f"Epoch {epoch + 1} completed.\n-------------------------------------------------\n")

        with open(loss_log_path, "a") as file:
            file.write("End of training\n")
    except Exception as e:
        with open(loss_log_path, "a") as file:
            file.write(f"Error occurred: {str(e)}\n")
            file.write(traceback.format_exc())

In [ ]:
# 모델을 불러와서 학습을 실행하는

from tensorflow.keras.models import load_model


# 경로 설정
data_dir = '/content/GANProject/spec512'
output_dir = 'drive/MyDrive/GANProject/GAN_0606'

# 저장된 모델 경로
generator_path = f'{output_dir}/generator_epoch_10'
discriminator_path = f'{output_dir}/discriminator_epoch_10'
# 하이퍼파라미터 설정
epochs = 200
batch_size = 8
save_interval = 5

# 모델 불러오기
generator = load_model(generator_path)
discriminator = load_model(discriminator_path)

noise_shape = (1024,)
input_shape = (512, 512, 1)

# Discriminator의 훈련 가능 상태 설정
discriminator.trainable = True

# Combined model 재구성
combined_input = Input(shape=noise_shape)
combined_output = discriminator(generator(combined_input))
combined = Model(combined_input, combined_output)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# 데이터셋 다시 로드 (필요시)
dataset = create_dataset(data_dir, batch_size)


# 학습 재개
train_gan(generator, discriminator, combined, dataset, epochs, batch_size, noise_shape, save_interval=10, output_dir=output_dir, start_epoch=10)

file_paths
tensor_slice
dataset.map
dataset.batch
train_gan started
Epoch 11 started


KeyboardInterrupt: 

In [ ]:
# 경로 설정
data_dir = '/content/GANProject/spec512'
output_dir = 'drive/MyDrive/GANProject/GAN_0607'

noise_shape = (1024,)
input_shape = (512, 512, 1)

# 하이퍼파라미터 설정
epochs = 200
batch_size = 8
save_interval = 5

# 모델 설정 및 컴파일
generator = build_generator(noise_shape)
generator.compile(loss='binary_crossentropy', optimizer=Adam(0.0005, 0.5))
discriminator = build_discriminator(input_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0001, 0.5), metrics=['accuracy'])

# Combined model 설정
discriminator.trainable = True
combined_input = Input(shape=noise_shape)
combined_output = discriminator(generator(combined_input))
combined = Model(inputs=combined_input, outputs=combined_output)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))


print("Generator model summary:")
generator.summary()
print("Discriminator model summary:")
discriminator.summary()
print("Discriminator weights: ", [var.name for var in discriminator.trainable_variables])
print("Combined model summary:")
combined.summary()

# 데이터셋 생성 및 훈련 시작
dataset = create_dataset(data_dir, batch_size)

train_gan(generator, discriminator, combined, dataset, epochs, batch_size, noise_shape, save_interval, output_dir, start_epoch=0)
print("GAN Main ends")

NameError: name 'build_generator' is not defined